In [5]:
import sys
from importlib import reload
import logging
import csv
from nltk.stem.porter import PorterStemmer
import string
import gensim
import pickle
from gensim.summarization.bm25 import BM25


porterStemmer = PorterStemmer()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def removeNonAscii(s):
    return "".join(filter(lambda x:ord(x) < 128, s))


def preprocess(passage):
    clean_passage = passage.lower()
    clean_passage = removeNonAscii(clean_passage)
    clean_passage = clean_passage.translate(str.maketrans('', '', string.punctuation)) 
    
    clean_passage_list = clean_passage.split()
    clean_passage_list_stem = []
    for word in clean_passage_list:
        if word not in gensim.parsing.preprocessing.STOPWORDS:
            clean_passage_list_stem.append(porterStemmer.stem(word))
    return clean_passage_list_stem

    
inputFile = "C:\\Users\\mapyredd\\Documents\\marco\\data\\collection.tsv"
corpus = []
count = 0
with open(inputFile, 'r', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    for row in reader:
        assert count == int(row[0])
        passage = row[1]
        corpus.append(preprocess(passage))
        count += 1
        if count % 1000000 == 0:
            print(count)


1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000


TypeError: a bytes-like object is required, not 'str'

In [21]:
with open("C:\\Users\\mapyredd\\Documents\\marco\\data\\collection_cleaned.tsv", "w") as fw:
    count = 0
    for line in corpus:
        fw.write(str(count) + "\t" + " ".join(line) + "\n")
        count += 1

In [10]:
bm25 = BM25(corpus)
bm25.initialize()

In [42]:
with open("C:\\Users\\mapyredd\\Documents\\marco\\data\\model\\bm25.pkl", 'wb') as fw:
    pickle.dump(bm25, fw, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
# with open("C:\\Users\\mapyredd\\Documents\\marco\\data\\model\\bm25.pkl", 'rb') as fr:
#     bm25 = pickle.load(fr)

average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys()) 
query = "manhattan project"
query_list = preprocess(query)
bm25.get_score(query_list, 7, average_idf)

14.422092756837039

In [45]:
from collections import defaultdict

queryFile = "C:\\Users\\mapyredd\\Documents\\marco\\data\\top1000.eval.tsv"
query_id_text = defaultdict(str)
query_doc_ids = defaultdict(list)
query_id_scores = defaultdict(list)
with open(queryFile, 'r', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    for row in reader:
        query_id_text[row[0]] = preprocess(row[2])
        query_doc_ids[row[0]].append(row[1])


In [46]:
for query_id, text in query_id_text.items():
    for doc_id in query_doc_ids[query_id]:
        score = bm25.get_score(text, int(doc_id), average_idf)
        query_id_scores[query_id].append((doc_id, score))

In [47]:
outputFile = "C:\\Users\\mapyredd\\Documents\\marco\\data\\output\\bm25.eval.tsv"
fw = open(outputFile, "w")
for query_id in query_id_scores.keys():
    query_id_scores[query_id] = sorted(query_id_scores[query_id], key=lambda x:x[1], reverse=True)[:10]
    for doc_id_score, rank in zip(query_id_scores[query_id], range(10)):
        fw.write(str(query_id) + "\t" + str(doc_id_score[0]) + "\t" + str(rank+1) + "\n")

In [48]:
len(query_id_scores)

6837

In [50]:
count = 0
inputFile = "C:\\Users\\mapyredd\\Documents\\marco\\data\\collection.tsv"
with open(inputFile, 'r', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    for row in reader:
        assert count == int(row[0])
        count += 1
        if count % 1000000 == 0:
            print(count)

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
